In [ ]:
%cd ..

In [ ]:
import simpy
import numpy as np
import matplotlib.pyplot as plt

# Simulation parameters
SIM_DURATION = 100  # seconds
ALERT_ARRIVAL_RATE = 2  # alerts/sec
ML_TIME_PER_ALERT = 3
INGEST_TIME = 1
FILTER_TIME = 1
NUM_ALERT_WORKERS = 1
NUM_ML_WORKERS = 3
NUM_FILTER_WORKERS = 3
NUM_UNIFIED_WORKERS = 6

np.random.seed(42)


# Metrics
class Metrics:
    def __init__(self):
        self.alert_latencies_3q = []
        self.alert_latencies_1q = []


# Scenario 1: 3-stage pipeline
def ingest_worker_3q(env, ingest_queue, ml_queue):
    while True:
        alert = yield ingest_queue.get()
        yield env.timeout(INGEST_TIME)
        yield ml_queue.put((alert[0], alert[1]))


def ml_worker_3q(env, ml_queue, filter_queue):
    while True:
        alert = yield ml_queue.get()
        yield env.timeout(ML_TIME_PER_ALERT)
        yield filter_queue.put(alert)


def filter_worker_3q(env, filter_queue, metrics: Metrics):
    while True:
        alert = yield filter_queue.get()
        yield env.timeout(FILTER_TIME)
        latency = env.now - alert[1]
        metrics.alert_latencies_3q.append(latency)


def alert_source_3q(env, ingest_queue):
    alert_id = 0
    while True:
        yield env.timeout(np.random.exponential(1 / ALERT_ARRIVAL_RATE))
        yield ingest_queue.put((alert_id, env.now))
        alert_id += 1


# Scenario 2: Unified workers
def unified_worker(env, alert_queue, metrics: Metrics):
    while True:
        alert = yield alert_queue.get()
        yield env.timeout(INGEST_TIME)
        yield env.timeout(ML_TIME_PER_ALERT)
        yield env.timeout(FILTER_TIME)
        latency = env.now - alert[1]
        metrics.alert_latencies_1q.append(latency)


def alert_source_1q(env, alert_queue):
    alert_id = 0
    while True:
        yield env.timeout(np.random.exponential(1 / ALERT_ARRIVAL_RATE))
        yield alert_queue.put((alert_id, env.now))
        alert_id += 1


# Run both simulations
metrics = Metrics()

# Scenario 1: 3-queue
env1 = simpy.Environment()
ingest_q = simpy.Store(env1)
ml_q = simpy.Store(env1)
filter_q = simpy.Store(env1)
for _ in range(NUM_ALERT_WORKERS):
    env1.process(ingest_worker_3q(env1, ingest_q, ml_q))
for _ in range(NUM_ML_WORKERS):
    env1.process(ml_worker_3q(env1, ml_q, filter_q))
for _ in range(NUM_FILTER_WORKERS):
    env1.process(filter_worker_3q(env1, filter_q, metrics))
env1.process(alert_source_3q(env1, ingest_q))
env1.run(until=SIM_DURATION)

# Scenario 2: Unified
env2 = simpy.Environment()
unified_q = simpy.Store(env2)
for _ in range(NUM_UNIFIED_WORKERS):
    env2.process(unified_worker(env2, unified_q, metrics))
env2.process(alert_source_1q(env2, unified_q))
env2.run(until=SIM_DURATION)

# Calculate throughputs
throughput_3q = len(metrics.alert_latencies_3q) / SIM_DURATION
throughput_1q = len(metrics.alert_latencies_1q) / SIM_DURATION
print(f"Throughput (3 Queues):   {throughput_3q:.2f} alerts/sec")
print(f"Throughput (1 Queue):    {throughput_1q:.2f} alerts/sec")

# Plot results
plt.figure(figsize=(10, 5))
plt.hist(metrics.alert_latencies_3q, bins=30, alpha=0.7, label="3 Queues")
plt.hist(
    metrics.alert_latencies_1q, bins=30, alpha=0.7, label="1 Queue (Unified)"
)
plt.xlabel("Alert Latency (s)")
plt.ylabel("Number of Alerts")
plt.title("Alert Latency Distribution (Simulated)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
